---

# Setting up some things

In [1]:
import matplotlib.pyplot as plt
import geopandas
import pandas as pd
import locale
import os
import os.path as osp
from typing import List, Dict, Set, Callable
import missingno as msno

HOME = "../../"

# the input directories
DATA = osp.join(HOME, "data")
CODE = osp.join(HOME, "code")
GREENSPACE_DATA = osp.join(DATA, "greenspace_data")
CENSUS_DATA = osp.join(DATA, "census_code_mappers")
STATES = geopandas.read_file('./state_shape_data/usa-states-census-2014.shp')
STATES.crs
STATES = STATES.to_crs("EPSG:3395")

# the output directories
def safe_mkdir(folder):
    if(not osp.exists(folder)):
        os.mkdir(folder)

CLEAN_DATA = osp.join(HOME, "clean_data")
CLEAN_GREENSPACE_DATA = osp.join(CLEAN_DATA, "greenspace_data")
safe_mkdir(CLEAN_DATA)
safe_mkdir(CLEAN_GREENSPACE_DATA)

In [2]:
def plot_on_US_map(fnc: Callable[[pd.Series], int], col_name: str):
    STATES[col_name] = STATES.apply(fnc, axis = 1)
    STATES.plot(column = col_name, legend = True)
    plt.title("How "+str(col_name)+" varies by state")
    plt.show()

Getting all the locational dataframes

In [3]:
blocks_df = pd.read_csv(osp.join(CENSUS_DATA, "us_blocks.csv"))
fips_df = pd.read_csv(osp.join(CENSUS_DATA, "fips_codes.csv"))
fips_df['geoid'] = fips_df.apply(lambda row: int(row.state_code+row.county_code), axis=1)
city_df = pd.read_csv(osp.join(CENSUS_DATA, "us_city_info.csv"))
zip_df = pd.read_csv(osp.join(CENSUS_DATA, "zcta_census_tract.csv"))

# Getting some dictionaries from the blocks_df
dc_block_geo_id_state_fip = dict(zip(blocks_df.block_geoid, blocks_df.state_fip))
dc_block_geo_id_county_fip = dict(zip(blocks_df.block_geoid, blocks_df.county_fip))
dc_block_geo_id_county_name = dict(zip(blocks_df.block_geoid, blocks_df.county_name))

# Getting dictionaries from the fips_df
dc_state_fip_to_state_abb = dict(zip(fips_df.state_code, fips_df.state))
dc_state_fip_to_state_name = dict(zip(fips_df.state_code, fips_df.state_name))
dc_county_fip_to_county_name = dict(zip(fips_df.county_code, fips_df.county))
dc_geoid_to_county_name = dict(zip(fips_df.geoid, fips_df.county))
dc_geoid_to_state_name = dict(zip(fips_df.geoid, fips_df.state_name))
dc_geoid_to_state_abb = dict(zip(fips_df.geoid, fips_df.state))

# Getting dictionaries from the city_df
dc_county_fips_to_state_abb =  dict(zip(city_df.county_fips, city_df.state_abbr))
dc_county_fips_to_state_name =  dict(zip(city_df.county_fips, city_df.state_name))
dc_county_fips_to_county_name =  dict(zip(city_df.county_fips, city_df.county_name))
dc_city_name_to_state_name =  dict(zip(city_df.city, city_df.state_name))

# Getting dictionaries from zip_df
dc_zcta10_to_state_fip =  dict(zip(zip_df.ZCTA10, zip_df.state_fip))
dc_zcta10_to_county_fip =  dict(zip(zip_df.ZCTA10, zip_df.county_fip))

---

# Cleaning the Percent Cover Data

There are 4 percent cover datasets to consider. We load them all.

In [4]:
pcc = pd.read_csv(osp.join(GREENSPACE_DATA, "percent_cover_county.txt"))
pctnb = pd.read_csv(osp.join(GREENSPACE_DATA, "percent_cover_tracts_no_buffer.txt"))
pctwb = pd.read_csv(osp.join(GREENSPACE_DATA, "percent_cover_tracts_with_buffer.txt"))
pczta = pd.read_csv(osp.join(GREENSPACE_DATA, "percent_cover_zipcode_tabulated_areas.txt"))
apc = [pcc, pctnb, pctwb, pczta]
apc_names = ["percent_cover_county", "percent_cover_tracts_no_buffer", "percent_cover_tracts_with_buffer", "percent_cover_zipcode_tabulated_areas"]

### Counting Nan Values

Turns out to have no NAN values initially

In [5]:
for pc in apc:
    print(pc.isnull().sum(axis = 0) / len(pc))

GEOID      0.0
NAME       0.0
pc_park    0.0
dtype: float64
GEOID       0.0
STATEFP     0.0
COUNTYFP    0.0
pc_park     0.0
dtype: float64
GEOID       0.0
STATEFP     0.0
COUNTYFP    0.0
pc_park     0.0
dtype: float64
ZCTA5CE10    0.0
pc_park      0.0
dtype: float64


## Counting Zero Values

Debatable if they need to be removed. Ask Yash once before going ahead.

In [6]:
for pc, pc_name in zip(apc, apc_names):
    print(pc_name + " " + str(len(pc[pc['pc_park'] == 0]) /(len(pc))))

percent_cover_county 0.01705276705276705
percent_cover_tracts_no_buffer 0.1731092668670214
percent_cover_tracts_with_buffer 0.03163858942898894
percent_cover_zipcode_tabulated_areas 0.18957650733380285


## Adding Other Columns
So that these dataframes are easily accessible

In [7]:
for pc in [pctnb, pctwb]:
    pc['state'] = pc['STATEFP'].map(dc_state_fip_to_state_abb)
    pc['state_name'] = pc['STATEFP'].map(dc_state_fip_to_state_name)    
    pc['county_name'] = pc['COUNTYFP'].map(dc_county_fip_to_county_name)

In [8]:
pcc['state'] = pcc['GEOID'].map(dc_county_fips_to_state_abb)
pcc['state_name'] = pcc['GEOID'].map(dc_county_fips_to_state_name)
pcc['county_name'] = pcc['GEOID'].map(dc_county_fips_to_county_name)

In [9]:
pczta['state_fip'] = pczta['ZCTA5CE10'].map(dc_zcta10_to_state_fip)
pczta['state_abb'] = pczta['state_fip'].map(dc_state_fip_to_state_abb)
pczta['state_name'] = pczta['state_fip'].map(dc_state_fip_to_state_name)
pczta['county_fip'] = pczta['ZCTA5CE10'].map(dc_zcta10_to_county_fip)
pczta['county_name'] = pczta['county_fip'].map(dc_county_fip_to_county_name)

In [10]:
for pc, nm in zip(apc, apc_names):
    tpc = pc.dropna()
    tpc.to_csv(osp.join(CLEAN_GREENSPACE_DATA, nm+str(".txt")), index=False)

---

# Cleaning the Urban Tree Canopy Dataframe

Just the one dataframe this time, however it is significantly more complicated than the percent cover datasets. Starting off by loading the dataset and plotting a heatmap

In [11]:
utcdf = pd.read_csv(osp.join(GREENSPACE_DATA, "urban_tree_canopy.csv"))

### Adding some columns

This dataset has the city name in a different format. Furthermore it does not have state or county so I am adding the same to the dataframe using the census block column. Furthermore I am taking rounding down the census block column as it should always be an int

In [12]:
utcdf = utcdf.astype({'census_block':'int'})
utcdf['state_fip'] = utcdf['census_block'].map(dc_block_geo_id_state_fip)
utcdf['state'] = utcdf['state_fip'].map(dc_state_fip_to_state_abb)
utcdf['state_name'] = utcdf['state_fip'].map(dc_state_fip_to_state_name)
utcdf['county_fip'] = utcdf['census_block'].map(dc_block_geo_id_county_fip)
utcdf['county_name'] = utcdf['census_block'].map(dc_block_geo_id_county_name)

In [14]:
blocks_df

,state_fip,county_fip,county_name,tract_fip,block,tract_geoid,block_geoid
0,4,1,Apache County,942600,1000,4001942600,40019426001000
1,4,1,Apache County,942600,1001,4001942600,40019426001001
2,4,1,Apache County,942600,1002,4001942600,40019426001002
3,4,1,Apache County,942600,1003,4001942600,40019426001003
4,4,1,Apache County,942600,1004,4001942600,40019426001004
...,...,...,...,...,...,...,...
11158380,41,71,Yamhill County,31000,2138,41071031000,410710310002138
11158381,41,71,Yamhill County,31000,2139,41071031000,410710310002139
11158382,41,71,Yamhill County,31000,2140,41071031000,410710310002140
11158383,41,71,Yamhill County,31000,2141,41071031000,410710310002141


### Counting Nan Values

After adding the state and county columns. 99.77% of the data is present

In [13]:
print(1- (utcdf.isnull().sum(axis = 0) / len(utcdf)))

city_name                  1.000000
census_block               1.000000
mean_percent_tree_cover    1.000000
tree_gap                   1.000000
surface_temp               1.000000
income_percent             1.000000
income_group               1.000000
pop_dens_group             1.000000
state_fip                  0.997709
state                      0.997709
state_name                 0.997709
county_fip                 0.997709
county_name                0.997709
dtype: float64


Washington DC seems to be missing from blocks_df. I think we can skip it is comparatively rare

In [15]:
utcdf[utcdf['state_fip'].isnull()]['city_name'].unique()

array(['Washington, DC--VA--MD'], dtype=object)

In [16]:
tutcdf = utcdf.dropna()
tutcdf.to_csv(osp.join(CLEAN_GREENSPACE_DATA, "urban_tree_canopy.csv"), index=False)